In [1]:
import os
import numpy as np
import pickle
import dimod
import dwave_token
import pandas as pd
import matplotlib.pyplot as plt
import time

from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
from dwave.system import DWaveCliqueSampler
from dwave.system import LeapHybridCQMSampler

# automatically generated embedding:
sampler = EmbeddingComposite(DWaveSampler(token=dwave_token.value))
clique_sampler = DWaveCliqueSampler(token=dwave_token.value)
hybrid_sampler = LeapHybridCQMSampler(token=dwave_token.value)

Matplotlib created a temporary cache directory at /tmp/matplotlib-1ggbxcdj because the default path (/home/user/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
def csv_to_matrix(file_name):
    dimension = len(np.genfromtxt(file_name)) - 1
    rows = np.array([element[0] for element in pd.read_csv(file_name, delimiter=';', usecols=[0]).to_numpy()])
    matrix = np.genfromtxt(file_name, delimiter=';', skip_header=1, usecols = range(1,dimension+1))
    return(rows, matrix)

def make_QUBO_upper_triangle(Q):
    Q = np.array(Q)
    return(np.transpose(Q - np.triu(Q)) + np.triu(Q))

def cyclic_permutate_array(arr,n):
    return(arr[len(arr)-n:] + arr[:len(arr)-n])
def distance_from_2Dpoints(points):
    n = len(points)
    d = np.zeros((n,n))
    for i in range(n):
        for j in range(i,n):
            d[i,j] = np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2)
    return(d)

def make_QUBO_upper_triangle(Q):
    Q = np.array(Q)
    return(np.transpose(Q - np.triu(Q)) + np.triu(Q))

def weighted_clustering_heuristic(distances, k, delta, n_max, solver='DWAVE', time_limit=0, shots=100, penalty=False, fix_variables=False, printornot=True, sort_samples=None):
    N = len(distances)
    def obj_0(values,g):
        return(sum(distances[i,j] * values[i,g] * values[j,g] for i in range(N) for j in range(i,N)))
    def N_cluster(values,g):
        N_g = sum(values[i,g] for i in range(N))
        return(N_g * (N_g-1))
    def obj(values):
        return(sum(obj_0(values,g)/N_cluster(values,g) if sum(values[i,g] for i in range(N)) > 1 else 0 for g in range(k)))
    def check_feasibility(values):
        return(np.array([sum(values[i,g] for g in range(k)) for i in range(N)]) == 1)
    def sampleset_to_values(sampleset, sort=None):
        if sort == 'lambda':
            prevalues_list = [dict(sampleset.samples()[i]) for i in range(len(sampleset))]
            if fix_variables:
                values_list = [{(i,g): int(g==0) if i == 0 else int(round(prevalues[g*N+i],0)) for g in range(k) for i in range(0,N)} for prevalues in prevalues_list]
            else:
                values_list = [{(i,g): int(round(pre_values[g*N+i],0)) for g in range(k) for i in range(N)} for prevalues in prevalues_list]
            lambda_list = [obj(values) for values in values_list]
            min_position = np.argmin(lambda_list)
            values = values_list[min_position]
        else:
            pre_values = dict(sampleset.lowest().samples()[0])
            if fix_variables:
                values = {(i,g): int(g==0) if i == 0 else int(round(pre_values[g*N+i],0)) for g in range(k) for i in range(0,N)}
            else:
                values = {(i,g): int(round(pre_values[g*N+i],0)) for g in range(k) for i in range(N)}
        return(values)
    if penalty == False and (solver != 'hybrid' and solver != 'CPLEX'):
        penalty = True
    if penalty == True:
        penalty = np.max(distances)* (N - k)
    if solver == 'CPLEX':
        model = pyo.AbstractModel()
        model.points = pyo.RangeSet(0, N-1)
        model.unique_pairs = pyo.Set(initialize=model.points*model.points, filter = lambda model, p1, p2: p1 < p2)
        model.clusters = pyo.RangeSet(0,k-1)
        model.d = pyo.Param(model.unique_pairs, initialize=lambda model, p1, p2: distances[p1,p2])
        model.lambda_n = pyo.Param(default=0, within=pyo.NonNegativeReals)
        model.x = pyo.Var(model.points*model.clusters, within=pyo.Binary)
        def obj_0_pyomo(model,g):
            return(sum(model.d[i,j]*model.x[i,g]*model.x[j,g] for i,j in model.unique_pairs))
        def N_cluster_pyomo(model,g):
            N_g = sum(model.x[i,g] for i in model.points)
            return(N_g * (N_g-1))
        def uniqueness(model,i):
            return(sum(model.x[i,g] for g in model.clusters) == 1)
        if penalty:
            model.obj = pyo.Objective(rule=lambda model: sum(obj_0_pyomo(model,g) - model.lambda_n * N_cluster_pyomo(model,g) for g in model.clusters) + penalty * sum((sum(model.x[i,g] for g in model.clusters) - 1)**2 for i in model.points))
        else:
            model.obj = pyo.Objective(rule=lambda model: sum(obj_0_pyomo(model,g) - model.lambda_n * N_cluster_pyomo(model,g) for g in model.clusters))
            model.uniqueness = pyo.Constraint(model.points, rule=uniqueness)
        opt = pyo.SolverFactory('cplex', tee=False)
        if time_limit != 0:
            opt.options['timelimit'] = time_limit
    else:
        QUBO_core = np.triu([[distances[i,j] if g == l else 0 for g in range(k) for j in range(N)] for l in range(k) for i in range(N)])
        bqm_core = dimod.BinaryQuadraticModel.from_qubo(QUBO_core)
        if solver != 'hybrid':
            QUBO_penalty = penalty * make_QUBO_upper_triangle([[(-1 if g == l else 1) if i == j else 0 for g in range(k) for i in range(N)] for l in range(k) for j in range(N)])
            bqm_core.add_linear_from_array(np.diag(QUBO_penalty))
            bqm_core.add_quadratic_from_dense(QUBO_penalty - np.diag(np.diag(QUBO_penalty)))
            bqm_core.offset += penalty*N
        if solver == 'clique':
            DWavesampler = clique_sampler
        elif solver == 'hybrid':
            def uniqueness(i):
                return([g*N + i for g in range(k)])
        else:
            DWavesampler = sampler
    lambdas = [0]
    ## start loop:
    for n in range(n_max+1):
        if printornot:
            print('n = ', n)
        lambda_n = lambdas[-1]
        if solver == 'CPLEX':
            instance = model.create_instance(data={None: {'lambda_n': {None: lambda_n}}})
            if fix_variables:
                for g in model.clusters:
                    instance.x[0,g].fix(int(g==0))
            results = opt.solve(instance)
            values = {(i,g): int(round(pyo.value(instance.x[i,g]),0)) for g in range(k) for i in range(N)}
        else:
            QUBO_lambda = - lambda_n * make_QUBO_upper_triangle([[1 if (g == l and i != j) else 0 for g in range(k) for i in range(N)] for l in range(k) for j in range(N)])
            bqm = dimod.as_bqm(bqm_core, copy=True)
            bqm.add_linear_from_array(np.diag(QUBO_lambda))
            bqm.add_quadratic_from_dense(QUBO_lambda - np.diag(np.diag(QUBO_lambda)))
            if fix_variables:
                bqm.fix_variables({g*N:int(g==0) for g in range(k)})
            if solver == 'hybrid':
                cqm = dimod.ConstrainedQuadraticModel.from_bqm(bqm)
                for i in range(int(fix_variables), N):
                    cqm.add_discrete(uniqueness(i), label='uniqueness' + str(i))
                sampleset = hybrid_sampler.sample_cqm(
                    cqm,
                    time_limit = time_limit,
                )
            else:
                chain_strength = max(bqm.quadratic.values())
                sampleset = DWavesampler.sample(
                    bqm,
                    num_reads=shots,
                    chain_strength=chain_strength
                )
            sampleset.resolve() # to reduce output readout time (does not seem to work tbh)
            values= sampleset_to_values(sampleset, sort=sort_samples)
        # post processing:
        feasibility = check_feasibility(values)
        while (not all(feasibility)) and penalty:
            if printornot:
                print('Post Processing...')
            for i in [i for i in range(N) if not feasibility[i]]:
                one_hot_sum = sum(values[i,g] for g in range(k))
                if one_hot_sum == 0:
                    values[(i,0)] = 1
                else:
                    for l in [l for l in range(k) if values[i,l] == 1][1:]:
                        values[(i,l)] = 0
            if solver != 'CPLEX':
                sampleset_feasible = sampleset.filter(lambda sample: all(np.array([sum(sample.sample[g*N+i] for g in range(k)) == 1 for i in range(int(fix_variables),N)])))
                if len(sampleset_feasible) > 0:
                    values_feasible= sampleset_to_values(sampleset_feasible, sort=sort_samples)
                    if obj(values_feasible) < obj(values):
                        if printornot:
                            print('Best feasible solution better than standard post processing')
                        values = values_feasible
                    elif printornot:
                        print('Standard post processing better than best feasible solution')
            feasibility = check_feasibility(values)
        lambda_n1 = obj(values)
        if all(lambda_n1 < np.array(lambdas[1:])) or n == 0:
            clusters = [[i for i in range(N) if values[(i,g)] == 1] for g in range(k)]
        lambdas += [lambda_n1]
        if printornot:
            print('lambda = ',lambda_n1)
        if abs(lambda_n1 - lambdas[-2]) <= delta: # break loop if algorithm terminated
            break
        elif any(abs((lambda_n1 - np.array(lambdas[1:-2]))) <= delta): # or if it is in a loop
            print('Algorithm cought in a loop')
            break
        elif n == n_max:
            print('Algorithm did not terminate in ' + str(n_max) + ' steps.')
    return(lambdas, clusters)

def elbow(distances, k_min, k_max, step, delta, n_max, solver='DWAVE', time_limit=0, penalty=False, fix_variables=False):
    objectives = []
    for k in range(k_min, k_max+1, step):
        print('k = ', k)
        lambdas, clusters = weighted_clustering_heuristic(distances, k, delta, n_max, solver, time_limit, penalty, fix_variables, printornot=False)
        objectives += [min(lambdas[1:])]
    plt.plot(range(k_min, k_max+1, step), objectives)
    plt.xlabel('k')
    plt.ylabel('objective')
    plt.show()
    return(objectives)

In [7]:
points = [(0,0),(0,1),(1,0),(1,1),(5,0),(5,1),(6,0),(6,1),(2,4),(2,5),(3,4),(3,5)]
# plt.scatter([x for x,y in points], [y for x,y in points])
# for i, p in enumerate(points):
#     plt.annotate(i, p)
# plt.show()
distances = distance_from_2Dpoints(points)
lambdas, clusters = weighted_clustering_heuristic(distances, k=3, delta=0, n_max=5, solver='', time_limit=0, shots=100, penalty=True, fix_variables=True, sort_samples='lambda')
print('lambdas:', lambdas)
print('objective: ', min(lambdas[1:]))
print('clusters:', clusters)
# plt.plot(lambdas)
# plt.show()

n =  0
lambda =  3.111114049942509
n =  1
Post Processing...
Standard post processing better than best feasible solution
lambda =  2.3561497254806927
n =  2
lambda =  1.9282429298968762
n =  3
lambda =  1.7071067811865475
n =  4
lambda =  2.2660648653200415
n =  5
Post Processing...
Standard post processing better than best feasible solution
lambda =  3.160629784651391
Algorithm did not terminate in 5 steps.
post_processing_time:  0.024326801300048828
lambdas: [0, 3.111114049942509, 2.3561497254806927, 1.9282429298968762, 1.7071067811865475, 2.2660648653200415, 3.160629784651391]
objective:  1.7071067811865475
clusters: [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]


In [10]:
folder = 'Compound data/'
file_name = 'Matrix_chemphys_withdrawn_tanimoto_real_values.csv'
rows, distance_J = csv_to_matrix(os.path.join(folder,file_name))
N = 30
start = 50
distances = np.array([row[:N] for row in distance_J[:N]])
# distances = np.array([[d**2 for d in row] for row in distances]) #square distances

In [11]:
lambdas, clusters = weighted_clustering_heuristic(distances, k=5, delta=1e-5, n_max=10, solver='clique', time_limit=5, shots=100, penalty=True, fix_variables=True, sort_samples=None)
print('Best result: ', min(lambdas[1:]))
print(clusters)

n =  0
Post Processing...
lambda =  0.8721559523809523
n =  1
Post Processing...
lambda =  0.685054761904762
n =  2
Post Processing...
lambda =  0.9880333333333334
n =  3
Post Processing...
lambda =  0.18123399014778316
n =  4
Post Processing...
lambda =  0.9493067765567766
n =  5
Post Processing...
lambda =  0.4831928571428571
n =  6
Post Processing...
lambda =  0.7197673992673994
n =  7
Post Processing...
lambda =  0.844081027667984
n =  8
Post Processing...
lambda =  0.9306376811594201
n =  9
Post Processing...
lambda =  0.4863532608695651
n =  10
Post Processing...
lambda =  0.8532277777777777
Algorithm did not terminate in 10 steps.
Best result:  0.18123399014778316
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], [29], [], [], []]
